# Práctica de Elastic / NoSQL

## Autor: Pedro Martínez

Para esta práctica se ha elegido como herramienta Elastic y como caso de uso se va a trabajar sobre el archivo de logs de un servidor web proporcionado. Para seleccionar esta opción he tenido en cuenta los siguientes motivos:

* La similitud con casos de uso reales en mi empresa, por lo que es de mucha utilidad conocer esta herramienta. 
* El gran número de variables que hay en el log, que permiten poder practicar diferentes opciones.

Algunas consideraciones para su funcionamiento:

* Estoy lanzando todos los comandos desde este notebook, no es necesario copiar y ejecutar en Kibana. La única excepción es el comando para iniciar Logstash, que se debe hacer en una terminal.
* El notebook lo estoy ejecutando desde una carpeta ***practica-PedroMartinez*** que cuelga de la carpeta ***work*** del material del módulo. En esta carpeta **practica-PedroMartinez** hago uso de las siguientes carpetas:
  * ***data***: En esta carpeta dejo el log descargado
  * ***pipeline***: En esta carpeta dejo el archivo de configuración de logstash
  * ***imagenes***: Imágenes que se muestran en este notebook

## Descripción del caso de uso acorde al conjunto de datos seleccionado. ¿Por qué es adecuado para realizar con el datastore seleccionado?

Para esta práctica he elegido Elastic y el archivo de log de un servidor web. En este log se está recogiendo la información de conexiones a un servidor web, con un gran número de variables, por lo que se puede obtener una valiosa información sobre el tipo de conexiones, dispositivos que la realizan, el estado de dichas conexiones, tamaño de los resultados y otras informaciones que pueden ser muy valiosas para entender el uso que está haciendo y mejorar el dimensionamiento y otros factores que puedan influir en el rendimiento.

En concreto, para esta práctica se van a evaluar los siguientes aspectos, partiendo de que se conoce la estructura del log, pero no su contenido. El objetivo es conseguir información sobre dicho contenido

* *Análisis de localizaciones:* ¿Desde dónde se están realizando las conexiones web que se recogen en el log? Para este caso utilizaremos la información geográfica contenida en la campo ***geo*** que se indexará como el tipo de datos correspondiente y se mostrará en el mapa que proporciona Kibana
* *Análisis de respuestas HTTP:* ¿Qué tasa de éxito tenemos en las conexiones? ¿Se producen muchos errores al acceder a la información? Para este caso se utilizará el campo ***response***
* *Análisis de extensiones de archivos descargados:* Queremos conocer qué tipos de archivos se descargan con más frecuencia, además del contenido de la página web. Para ello se utilizará el campo ***extension***
* *Análisis de agents:* ¿Se puede optimizar la presentación de la página web en función del agent del dispositivo desde el que se realiza la conexión? Queremos saber cuáles son los navegadores más usados
* *Análisis de histograma de bytes:* Para dimensionar el ancho de banda, es interesante saber el tamaño en bytes de las respuestas, para lo que utilizaremos el campo ***bytes***
* *Análisis de dispositivos:* ¿Qué dispositivos utilizan los usuarios? en el campo ***machine*** podemos ver el sistema operativo y la RAM, que nos puede ayudar a optimizar las consultas


## Modelo de datos 

El archivo de logs proporcionado está en formato jsonl, que es el utilizado para realizar logging de muchas líneas en formato json. El formato json, nativo para Elasticsearch, se puede indexar con mucha facilidad, y los campos se extraen de forma automática.

Para esta práctica he decidido utilizar Logstash, y un pipeline en el que se van a procesar las líneas una a una extrayendo los campos y dándoles la estructura deseada. Para ello, utilizo un patrón de grok que irá obteniendo cada campo. Se incluye también la información para realizar la práctica con un processor de json, pero esta no es la opción elegida.

La información contendida en el archivo de log corresponde en su totalidad a cada petición realizada al servidor web. Para modelar los datos he separado esta información en valores que pertenecen a la petición, y valores que indican más el resultado y el estado de dicha petición. El documento que se inserta en Elasticsearch, una vez procesado por Logstash, tiene la siguiente estructura:

<img src="./imagenes/modelo-datos-json.jpg" alt="modelo datos json"/>

Por un lado se han dejado los valores como ***estado_peticion***, ***nivel_peticion*** o ***dispositivo_clilente*** en el raíz del documento. Y colgando del campo ***datos_peticion*** se han dejado el resto de valores, como ***coordenadas_geograficas***, ***clientip***, o ***url***

## Creación del índice en Elasticsearch

A continuación se muestra la estructura utilizada para crear el índice en Elasticsearch. Los valores básicos son los siguientes:
* nombre del índice: practica-final
* número de shards: 3
* número de réplicas: 2

`
{
  "settings" : {
    "index" : {
      "number_of_shards" : 3,
      "number_of_replicas" : 2
    }
  }, 
  "mappings": {
    "properties": {
      "estado_peticion": {
        "type": "keyword"
      },
      "event":  {
        "properties": {
          "ingested": {
            "type": "date"
          }
        }
      },
      "host": {
        "type": "keyword"
      },
      "nivel_peticion": {
        "type": "keyword"
      },
      "version_mensaje": {
        "type": "integer"
      },
      "dispositivo_cliente": {
        "properties": {
          "os": {
            "type": "keyword"
          },
          "ram": {
            "type": "long"
          }
        }
      },
      "datos_peticion": {
        "properties": {
          "agent": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "bytes": {
            "type": "long"
          },
          "clientip": {
            "type": "ip"
          },
          "extension": {
            "type": "keyword"
          },
          "geo": {
            "properties": {
              "coordenadas_geograficas": {
                "type": "geo_point"
              },
              "src": {
                "type": "keyword"
              },
              "dest": {
                "type": "keyword"
              },
              "srcdest": {
                "type": "keyword"
              }
            }
          },
          "headings": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "host": {
            "type": "keyword"
          },
          "ip_origen": {
            "type": "ip"
          },
          "links": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "memoria": {
            "properties": {
              "memory": {
                "type": "integer"
              },
              "phpmemory": {
                "type": "integer"
              }
            }
          },
          "message": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "referer": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "relatedContent": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "request": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "respuesta": {
            "type": "integer"
          },
          "timestamp_peticion": {
            "type": "date"
          },
          "url": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "utc_time": {
            "type": "date"
          }
        }
      }
    }
  }
}
`

Para poder realizar las búsquedas cuando los datos estén insertados, se han definido las variables de la siguiente manera (no todas las variables se utilizan en esta práctica, pero pueden ser útiles para obtener otra información):

* keyword:
    * estado_peticion, host, nivel_peticion, dispositivo_cliente.os
    * datos_peticion.agent, datos_peticion.extension, datos_peticion.geo.src, datos_peticion.geo.dest, datos_peticion.geo.destsrc, datos_peticion.headings, datos_peticion.host, datos_peticion.links, datos_peticion.message, datos_peticion.referer, datos_peticion.relatedContent, datos_peticion.url
    
* date:
    * event.ingested, datos_peticion.timestamp_peticion, datos_peticion.utc_time

* integer: 
    * version_mensaje, datos_peticion.memoria.memory, datos_peticion.memoria.phpmemory, respuesta
    
* long
    * dispositivo_cliente.ram, datos_peticion.bytes
    
* text:
    * datos_peticion.agent, datos_peticion.headings, datos_peticion.message, datos_peticion.referer, datos_peticion.url
    
* ip: 
    * datos_peticion.clientip, datos_peticion.ip_origen
* geo_point:
    * datos_peticion.geo.coordenadas_geograficas

## Procesado en Logstash

Para procesar los logs se utiliza un pipeline que se ejecuta desde este notebook, y el archivo de configuración se que se pasa en el pipeline al levantar docker.

El contenido del archivo de configuración ***practica-final.conf*** es el siguiente:


<img src="./imagenes/logstash-practica-final-conf.jpg" alt="logstash-practica-final-conf"/>

 

Se mantiene la estructura del utilizado en el curso, y se indica cuales son el índice y el pipeline a utilizar.

La configuración del pipeline de ingesta es el siguiente:

`
{
  "description" : "Pipeline práctica Elastic",
  "processors" : [
    {
      "set" : {
        "field" : "event.ingested",
        "value" : "{{_ingest.timestamp}}"
      }
    },
    {
      "grok" : {
        "patterns" : [
          "{\"@timestamp\":\"%{TIMESTAMP_ISO8601:timestamp}\",\"ip\":\"%{IP:datos_peticion.ip_origen}\",\"extension\":\"%{DATA:datos_peticion.extension}\",\"response\":\"%{NUMBER:datos_peticion.respuesta}\",\"geo\":{\"coordinates\":{\"lat\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lat},\"lon\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lon}},\"src\":\"%{DATA:datos_peticion.geo.src}\",\"dest\":\"%{DATA:datos_peticion.geo.dest}\",\"srcdest\":\"%{DATA:datos_peticion.geo.srcdest}\"},\"@tags\":\\[\"%{WORD:estado_peticion}\",\"%{WORD:nivel_peticion}\"\\],\"utc_time\":\"%{TIMESTAMP_ISO8601:datos_peticion.utc_time}\",\"referer\":\"%{DATA:datos_peticion.referer}\",\"agent\":\"%{DATA:datos_peticion.agent}\",\"clientip\":\"%{IP:datos_peticion.clientip}\",\"bytes\":%{NUMBER:datos_peticion.bytes},\"host\":\"%{URIHOST:datos_peticion.host}\",\"request\":\"%{DATA:datos_peticion.request}\",(\"memory\":%{NUMBER:datos_peticion.memoria.memory},|)(\"phpmemory\":%{NUMBER:datos_peticion.memoria.phpmemory},|)\"url\":\"%{URI:datos_peticion.url}\",\"@message\":\"%{DATA:datos_peticion.message}\",\"spaces\":\"%{DATA:datos_peticion.spaces}\",\"xss\":\"%{DATA:datos_peticion.xss}\",\"headings\":\\[\"%{DATA:datos_peticion.headings}\"\\],\"links\":\\[%{DATA:datos_peticion.links}\\],\"relatedContent\":\\[%{DATA:datos_peticion.relatedContent}\\],\"machine\":{(\"os\":\"%{DATA:dispositivo_cliente.os}\",|)\"ram\":%{NUMBER:dispositivo_cliente.ram:long}},\"@version\":\"%{NUMBER:version_mensaje}\"}"
        ],
        "ignore_missing" : true,
        "field" : "message"
      }
    },
    {
      "remove" : {
        "field" : "message"
      }
    },
    {
      "remove" : {
        "field" : "datos_peticion.spaces"
      }
    },
    {
      "remove" : {
        "field" : "event.original"
      }
    },
    {
      "set" : {
        "field" : "host",
        "value" : "PRACTICA_PEDRO_M"
      }
    },
    {
        "split": {
          "description": "Divido el campo headings",
          "field": "datos_peticion.headings",
          "separator": "\",\"",
          "ignore_missing": true,
          "tag": "split-headings"
      }
    },
    {
        "split": {
          "description": "Divido el campo links",
          "field": "datos_peticion.links",
          "separator": "\",\"",
          "ignore_missing": true,
          "tag": "split-links"
      }
    }
  ],
  "on_failure" : [
    {
      "set" : {
        "field" : "error.message",
        "value" : "{{ _ingest.on_failure_message }}"
      }
    }
  ]
}
`

* Se borran los campos que almacenan el mensaje de log original, ya que lo tenemos parseado y no son necesarios. Se borra también un campo que no es de utilidad: ***spaces***.
* Se incluye como valor de host que indexa los logs el valor PRACTICA_PEDRO_M
* Los campos heading y link son listas de valores, por lo que se hace un split para separar los valores, ya que esto no lo hago con grok

A continuación se muestra el patrón de grok con el que se extraen los valores

`
{\"@timestamp\":\"%{TIMESTAMP_ISO8601:timestamp}\",
\"ip\":\"%{IP:datos_peticion.ip_origen}\",
\"extension\":\"%{DATA:datos_peticion.extension}\",
\"response\":\"%{NUMBER:datos_peticion.respuesta}\",
\"geo\":{\"coordinates\":{\"lat\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lat},
\"lon\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lon}},
\"src\":\"%{DATA:datos_peticion.geo.src}\",
\"dest\":\"%{DATA:datos_peticion.geo.dest}\",
\"srcdest\":\"%{DATA:datos_peticion.geo.srcdest}\"},
\"@tags\":\\[\"%{WORD:estado_peticion}\",\"%{WORD:nivel_peticion}\"\\],
\"utc_time\":\"%{TIMESTAMP_ISO8601:datos_peticion.utc_time}\",
\"referer\":\"%{DATA:datos_peticion.referer}\",
\"agent\":\"%{DATA:datos_peticion.agent}\",
\"clientip\":\"%{IP:datos_peticion.clientip}\",
\"bytes\":%{NUMBER:datos_peticion.bytes},
\"host\":\"%{URIHOST:datos_peticion.host}\",
\"request\":\"%{DATA:datos_peticion.request}\",
(\"memory\":%{NUMBER:datos_peticion.memoria.memory},|)
(\"phpmemory\":%{NUMBER:datos_peticion.memoria.phpmemory},|)
\"url\":\"%{URI:datos_peticion.url}\",
\"@message\":\"%{DATA:datos_peticion.message}\",
\"spaces\":\"%{DATA:datos_peticion.spaces}\",
\"xss\":\"%{DATA:datos_peticion.xss}\",
\"headings\":\\[\"%{DATA:datos_peticion.headings}\"\\],
\"links\":\\[%{DATA:datos_peticion.links}\\],
\"relatedContent\":\\[%{DATA:datos_peticion.relatedContent}\\],
\"machine\":{(\"os\":\"%{DATA:dispositivo_cliente.os}\",|)
\"ram\":%{NUMBER:dispositivo_cliente.ram:long}},
\"@version\":\"%{NUMBER:version_mensaje}\"}
`

Tras un análisis de los logs, se ve que algunos campos no aparecen siempre, por lo que se ponen entre paréntesis para indicar que son opcionales.

### Parseo de json directamente con Logstash

Como se comentaba anteriormente, con Logstash se puede parsear directamente cada documento json, y se extraen los campos directamente, sin tener que conocerlos previamente. De esta manera, se habría parseado correctamente, por ejemplo, el campo relatedContent, que es una lista de documentos json, que con mi patrón de grok no se extran con este formato.

El archivo de configuración de logstash hubiera sido el siguiente, y no se habría utilzado patrón de grok:


<img src="./imagenes/logstash-practica-final-conf-json.jpg" alt="logstash-practica-final-conf-json"/>



# Ejecución de la práctica

## Índices y mappings

El primer paso es crear el índice con sus mappings:

In [5]:
! curl -X PUT "http://elasticsearch:9200/practica-final?pretty" -H 'Content-Type: application/json' -d' \
{ \
  "settings" : { \
    "index" : { \
      "number_of_shards" : 3, \
      "number_of_replicas" : 2 \
    } \
  },  \
  "mappings": { \
    "properties": { \
      "estado_peticion": { \
        "type": "keyword" \
      }, \
      "event":  { \
        "properties": { \
          "ingested": { \
            "type": "date" \
          } \
        } \
      }, \
      "host": { \
        "type": "keyword" \
      }, \
      "nivel_peticion": { \
        "type": "keyword" \
      }, \
      "version_mensaje": { \
        "type": "integer" \
      }, \
      "dispositivo_cliente": { \
        "properties": { \
          "os": { \
            "type": "keyword" \
          }, \
          "ram": { \
            "type": "long" \
          } \
        } \
      }, \
      "datos_peticion": { \
        "properties": { \
          "agent": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "bytes": { \
            "type": "long" \
          }, \
          "clientip": { \
            "type": "ip" \
          }, \
          "extension": { \
            "type": "keyword" \
          }, \
          "geo": { \
            "properties": { \
              "coordenadas_geograficas": { \
                "type": "geo_point" \
              }, \
              "src": { \
                "type": "keyword" \
              }, \
              "dest": { \
                "type": "keyword" \
              }, \
              "srcdest": { \
                "type": "keyword" \
              } \
            } \
          }, \
          "headings": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "host": { \
            "type": "keyword" \
          }, \
          "ip_origen": { \
            "type": "ip" \
          }, \
          "links": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "memoria": { \
            "properties": { \
              "memory": { \
                "type": "integer" \
              }, \
              "phpmemory": { \
                "type": "integer" \
              } \
            } \
          }, \
          "message": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "referer": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "relatedContent": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "request": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "respuesta": { \
            "type": "integer" \
          }, \
          "timestamp_peticion": { \
            "type": "date" \
          }, \
          "url": { \
            "type": "text", \
            "fields": { \
              "keyword": { \
                "type": "keyword", \
                "ignore_above": 256 \
              } \
            } \
          }, \
          "utc_time": { \
            "type": "date" \
          } \
        } \
      } \
    } \
  } \
}'

{
  "error" : {
    "root_cause" : [
      {
        "type" : "resource_already_exists_exception",
        "reason" : "index [practica-final/ceHFqPGYQ_ahE13Z2rXMfw] already exists",
        "index_uuid" : "ceHFqPGYQ_ahE13Z2rXMfw",
        "index" : "practica-final"
      }
    ],
    "type" : "resource_already_exists_exception",
    "reason" : "index [practica-final/ceHFqPGYQ_ahE13Z2rXMfw] already exists",
    "index_uuid" : "ceHFqPGYQ_ahE13Z2rXMfw",
    "index" : "practica-final"
  },
  "status" : 400
}


## Pipeline de ingesta

A continuación, creo el pipeline de ingesta, con el patrón neceario para parsear los logs:

In [6]:
!curl -X PUT http://elasticsearch:9200/_ingest/pipeline/practica-final -H 'Content-Type: application/json' -d ' \
{ \
  "description" : "Pipeline práctica de Elastic", \
  "processors" : [ \
    { \
      "set" : { \
        "field" : "event.ingested", \
        "value" : "{{_ingest.timestamp}}" \
      } \
    }, \
    { \
      "grok" : { \
        "patterns" : [ \
          "{\"@timestamp\":\"%{TIMESTAMP_ISO8601:timestamp}\",\"ip\":\"%{IP:datos_peticion.ip_origen}\",\"extension\":\"%{DATA:datos_peticion.extension}\",\"response\":\"%{NUMBER:datos_peticion.respuesta}\",\"geo\":{\"coordinates\":{\"lat\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lat},\"lon\":%{NUMBER:datos_peticion.geo.coordenadas_geograficas.lon}},\"src\":\"%{DATA:datos_peticion.geo.src}\",\"dest\":\"%{DATA:datos_peticion.geo.dest}\",\"srcdest\":\"%{DATA:datos_peticion.geo.srcdest}\"},\"@tags\":\\[\"%{WORD:estado_peticion}\",\"%{WORD:nivel_peticion}\"\\],\"utc_time\":\"%{TIMESTAMP_ISO8601:datos_peticion.utc_time}\",\"referer\":\"%{DATA:datos_peticion.referer}\",\"agent\":\"%{DATA:datos_peticion.agent}\",\"clientip\":\"%{IP:datos_peticion.clientip}\",\"bytes\":%{NUMBER:datos_peticion.bytes},\"host\":\"%{URIHOST:datos_peticion.host}\",\"request\":\"%{DATA:datos_peticion.request}\",(\"memory\":%{NUMBER:datos_peticion.memoria.memory},|)(\"phpmemory\":%{NUMBER:datos_peticion.memoria.phpmemory},|)\"url\":\"%{URI:datos_peticion.url}\",\"@message\":\"%{DATA:datos_peticion.message}\",\"spaces\":\"%{DATA:datos_peticion.spaces}\",\"xss\":\"%{DATA:datos_peticion.xss}\",\"headings\":\\[\"%{DATA:datos_peticion.headings}\"\\],\"links\":\\[%{DATA:datos_peticion.links}\\],\"relatedContent\":\\[%{DATA:datos_peticion.relatedContent}\\],\"machine\":{(\"os\":\"%{DATA:dispositivo_cliente.os}\",|)\"ram\":%{NUMBER:dispositivo_cliente.ram:long}},\"@version\":\"%{NUMBER:version_mensaje}\"}" \
        ], \
        "ignore_missing" : true, \
        "field" : "message" \
      } \
    }, \
    { \
      "remove" : { \
        "field" : "message" \
      } \
    }, \
    { \
      "remove" : { \
        "field" : "datos_peticion.spaces" \
      } \
    }, \
    { \
      "remove" : { \
        "field" : "event.original" \
      } \
    }, \
    { \
      "set" : { \
        "field" : "host", \
        "value" : "PRACTICA_PEDRO_M" \
      } \
    }, \
    { \
        "split": { \
          "description": "Divido el campo headings", \
          "field": "datos_peticion.headings", \
          "separator": "\",\"", \
          "ignore_missing": true, \
          "tag": "split-headings" \
      } \
    }, \
    { \
        "split": { \
          "description": "Divido el campo links", \
          "field": "datos_peticion.links", \
          "separator": "\",\"", \
          "ignore_missing": true, \
          "tag": "split-links" \
      } \
    } \
  ], \
  "on_failure" : [ \
    { \
      "set" : { \
        "field" : "error.message", \
        "value" : "{{ _ingest.on_failure_message }}" \
      } \
    } \
  ] \
}'

{"acknowledged":true}

## Ejecución de logstash

A continuación se muestra el comando docker utilizado para arrancar logstash en el entorno de desarrollo de la práctica. Es necesario realizarlo en un terminal aparte. 

Es necesario esperar a que arranque completamente Logstash

In [7]:
docker run --rm -it --network=datahack-elastic-v2_default -v \
"/datos/Datahack/Módulo BC6 - ELK/datahack-elastic-v2/work/practica-PedroMartinez/pipeline/":/usr/share/logstash/pipeline/ \
-v "/datos/Datahack/Módulo BC6 - ELK/datahack-elastic-v2/work/practica-PedroMartinez/data/":/tmp/data/ \
docker.elastic.co/logstash/logstash:8.3.3

SyntaxError: invalid syntax (<ipython-input-7-8fb911f8f689>, line 1)

## Entorno Python

Preparación para poder utilizar Python con Elasticsearch:

In [8]:
# Instalación de librerías necesarias
!pip install certifi==2021.5.30
!pip install elasticsearch==7.13.1
!pip install urllib3

  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2018.8.24
ERROR: Cannot uninstall 'certifi'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [9]:
# Importo módulos necesarios
import requests
from elasticsearch import Elasticsearch

# Conecto con la instancia de elasticsearch
es_host = 'http://elasticsearch:9200'
es = Elasticsearch([es_host])

In [10]:
# Descargo el archivo de logs
response = requests.get('https://github.com/rafaelgarrote/datahack-nosql/raw/nosql-especial/workespecial/practica/data/logs.jsonl')

# Lo almaceno en la carpeta datos que cuelga de la actual
with open("./data/logs-practica.jsonl", mode="wb") as file:
    file.write(response.content)

#### NOTA: 
*Durante la realización de la práctica no ha habido problemas en la descarga del archivo de log. Solamente ha sido necesario tocar permisos en la carpeta de descarga para que se pueda escribir el contenido. Al ser un archivo de tamaño grande, no se  incluye en la entrega. El ejercicio entero depende de que se descargue correctamente, por favor si no funcionara, copiar a mano en la carpeta ***data*** dentro de la carpeta de la práctica*

Una vez descargado el archivo, con Logstash corriendo, se empiezan a indexar las líneas del log de acuerdo a los patrones configurados. En la ventana de Logstash se muestra un punto por cada documento procesado.

A continuación se comprueba que se han insertado datos en Elasticsearch, una vez descargado el archivo. Es necesario esperar a que se inserten todos. Cuando termina, el valor es 14005

In [12]:
# Realizamos la cuenta de documentos en el índice practica-final y se muestra por pantalla
cuenta = es.count(index='practica-final')["count"]

print (f"El número de documentos en el índice es: {cuenta}")

El número de documentos en el índice es: 585


## Búsquedas

### Análisis de localizaciones

Una vez insertados los datos, podemos ver en Kibana la representación en un mapa de los valores incluidos en el campo geo. Se comprueba que en el log disponible todas las conexiones corresponden a Estados Unidos:

<img src="./imagenes/kibana-coordenadas.jpg" alt="mapa coordenadas"/>

Además, haciendo zoom, se puede apreciar que el número de conexiones coincide con la densidad de polación del país. con poca población en Alaska, las islas y el oeste, y una mayor densidad al este:

<img src="./imagenes/kibana-coordenadas-2.jpg" alt="mapa coordenadas ampliado"/>


* Análisis de respuestas HTTP

Vamos a obtener todos los valores diferentes que se encuentran en el dataset para poder extraer alguna conclusión:



In [13]:
search_dict = {
    "aggs" : {
        "respuestas" : {
            "terms" : { "field" : "datos_peticion.respuesta" } 
        }
    }
    
}

respuesta = es.search(index="practica-final", body=search_dict)

# Se imprime por pantalla el resultado de la consulta
for elemento in respuesta['aggregations']['respuestas']['buckets']:
    print (f"El código de respuesta {elemento['key']} aparece {elemento['doc_count']} veces")

El código de respuesta 200 aparece 4174 veces
El código de respuesta 404 aparece 234 veces
El código de respuesta 503 aparece 149 veces


Como se aprecia en los resultados, el código 200 (OK) es el que aparece en más ocasiones. A continuación va el código 404 (página no encontrada) y el 503 (servicio no disponible) 

* Análisis de extensiones de archivos descargados

Con la siguiente búsqueda queremos saber qué tipos de archivos son los más descargados en este servidor web (en el rango recogido en el log)

In [14]:
search_dict = {
    "aggs" : {
        "extensiones" : {
            "terms" : { "field" : "datos_peticion.extension" } 
        }
    }
    
}

respuesta = es.search(index="practica-final", body=search_dict)

# Se imprime por pantalla el resultado de la consulta
for elemento in respuesta['aggregations']['extensiones']['buckets']:
    print (f"La extensión de archivo {elemento['key']} aparece {elemento['doc_count']} veces")

La extensión de archivo jpg aparece 4910 veces
La extensión de archivo css aparece 1224 veces
La extensión de archivo png aparece 718 veces
La extensión de archivo gif aparece 481 veces
La extensión de archivo php aparece 208 veces


Vemos en los resultados que los archivos descargados son principalmente imágenes y archivos asociados al diseño web. Principalmente se descargan imágenes jpg. En una búsqueda posterior veremos qué tamaños suelen tener estos archivos.

* Análisis de histograma de bytes

Para analizar la distribución de valores de bytes descargados, se va a utilizar la opción de histograma. Tras diferentes pruebas, se considera que 1024 bytes es un buen valor para hacer los intervalos:

In [15]:
search_dict = {
    "aggs" : {
        "bytes" : {
            "histogram" : {
                "field" : "datos_peticion.bytes",
                "interval" : 1024
            }
        }
    }
}
response = es.search(index="practica-final", body=search_dict)

# Se imprime por pantalla el resultado de la consulta
for i in response['aggregations']['bytes']['buckets']:
    print(i['key'], i['doc_count'])

0.0 831
1024.0 482
2048.0 936
3072.0 890
4096.0 926
5120.0 900
6144.0 913
7168.0 881
8192.0 922
9216.0 681
10240.0 45
11264.0 45
12288.0 52
13312.0 45
14336.0 50
15360.0 49
16384.0 38
17408.0 42
18432.0 45
19456.0 19


Se aprecia en los resultados que por debajo de 10KB la distribucióin es más o menos uniforme, y por encima de este valor los archivos también se distribuyen de manera uniforme con un tamaño considerablemente menor.

* Análisis de agents

De nuevo, con una agregación, se pueden obtener los diferentes agents utilizados para saber qué navegadores se utilizan al realizar las peticiones web:

In [16]:
search_dict = {
    "aggs" : {
        "agents" : {
            "terms" : { "field" : "datos_peticion.agent.keyword" } 
        }
    }
    
}

respuesta = es.search(index="practica-final", body=search_dict)

# print the request's response to terminal
for elemento in respuesta['aggregations']['agents']['buckets']:
    print (f"El agent {elemento['key']}\n\t aparece {elemento['doc_count']} veces")

El agent Mozilla/5.0 (X11; Linux x86_64; rv:6.0a1) Gecko/20110421 Firefox/6.0a1
	 aparece 3651 veces
El agent Mozilla/5.0 (X11; Linux i686) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/11.0.696.50 Safari/534.24
	 aparece 3203 veces
El agent Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)
	 aparece 2784 veces


Los resultados se reparten en 3 valores, sin grandes diferencias entre ellos. Parece que los dos más utilizados están basados en Linux, uno de ellos de Apple, y el menos utilizado es una versión de Windows.

* Análisis de dispositivos

A continuación vamos a analizar los dispositivos, y se podrá comprobar si los resultados obtenidos en el apartado anterior son coherentes con los que se obtengan aquí:

In [17]:
search_dict = {
    "aggs" : {
        "dispositivos_os" : {
            "terms" : { "field" : "dispositivo_cliente.os" } 
        }
    }
    
}

respuesta = es.search(index="practica-final", body=search_dict)

# print the request's response to terminal
for elemento in respuesta['aggregations']['dispositivos_os']['buckets']:
    print (f"El agent {elemento['key']}\n\t aparece {elemento['doc_count']} veces")

El agent win 7
	 aparece 2055 veces
El agent win 8
	 aparece 2038 veces
El agent ios
	 aparece 2032 veces
El agent win xp
	 aparece 2001 veces
El agent osx
	 aparece 1017 veces


Los resultados muestran que los 3 dispositivos más utilizados son con diferentes versiones de Windows (7, 8 y XP), y a continuación IOS y OSX. Esto no coincide con lo visto en los Agents. Por tanto, no se puede asegurar con el agent el dispositivo utilizado. 

A continuación veremos la memoria RAM de los dispositivos utilizados. Para ello sacaremos los datos utilizando rangos: hasta 8 GB, de 8 a 16GB, y a partir de 16GB.

In [18]:
search_dict = {
    "aggs" : {
        "dispositivos_ram" : {
            "range" : {
                "field" : "dispositivo_cliente.ram",
                "ranges" : [
                    { "key": "hasta 8GB", "to" : 8000000000 },
                    { "key": "de 8GB a 16GB", "from" : 8000000000, "to" : 16000000000 },
                    { "key": "de 16GB a 32GB", "from" : 16000000000, "to" : 32000000000 },
                    { "key": "a partir de 16GB", "from" : 32000000000 }
                ]
            }
        }
    }
}
response = es.search(index="practica-final", body=search_dict)

# print the request's response to terminal
for elemento in response['aggregations']['dispositivos_ram']['buckets']:
    print (f"Para el rango '{elemento['key']}' se encuentran {elemento['doc_count']} dispositivos")

Para el rango 'hasta 8GB' se encuentran 3113 dispositivos
Para el rango 'de 8GB a 16GB' se encuentran 4105 dispositivos
Para el rango 'de 16GB a 32GB' se encuentran 3539 dispositivos
Para el rango 'a partir de 16GB' se encuentran 590 dispositivos


Con estos resultados vemos que en general los equipos disponen de memoria RAM suficiente para realizar consultas web sin problemas. Un buen número de dispositivos tiene menos de 8GB de RAM, pero la mayoría se mueve entre los 8 y los 32 GB.

# Conclusiones

## ¿Qué te parece la base de datos seleccionada como data store?

La base de datos elegida como datastore para el dataset proporcionado es, creo, una buena elección, ya que el software de Elastic es perfecto para este caso de uso. Indexar logs y obtener información sobre su contenido es uno de los principales puntos fuertes de Elastic. El hecho de que esté orientado a búsquedas temporales hace que sea muy útil para entender lo que pasó en cierto período de tiempo.


## ¿Qué te ha parecido el ejercicio?

El ejercicio me ha parecido muy interesante y útil. El caso de uso que he elegido es muy útil en mi día a día y a buen seguro que podré sacarle partido en el futuro. 
Como usuario de Splunk, herramienta que utilizo a diario y que tiene bastante parecido con el stack de Elastic, sé que podré aprovechar todo lo que he tenido que hacer.


## ¿Qué has aprendido?

He aprendido principalmente a analizar el contenido de un archivo de log para ver qué información útil se puede sacar. Y a partir de este análisis, poder parsear e insertar la información en Elastic para realizar búsquedas que den utilidad a estos datos. En general, he aprendido a utilizar y poner a funcionar conjuntamente las  herramientas Elasticsearch, Logstash y Kibana.


## ¿Qué has echado de menos?

He echado de menos conocer en profundidad la herramienta Beats para poder simular que el log estaba en una máquina real, y enviarlo a Elasticsearch de forma automática.

## ¿Cómo mejorarías la práctica?

La práctica la mejoraría en los siguientes puntos:
* utilizando Beats, como he comentado en el apartado anterior.
* parseando el log utilizando el procesador de json de logstash. De esta manera, podría haber extraído información también de las etiquetas de Twitter.
* realizando un dashboard en Kibana donde se pudiera ver toda la información de las búsquedas en un único sitio
